In [9]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 120)
START = "2022-01-01"
END = "2025-12-18"

In [12]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.0 MB 3.3 MB/s eta 0:00:01
     ----------------- ---------------------- 1.3/3.0 MB 3.6 MB/s eta 0:00:01
     --------------------------- ------------ 2.1/3.0 MB 3.6 MB/s eta 0:00:01
     -------------------------------------- - 2.9/3.0 MB 3.5 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 3.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------------------- ---------

In [13]:
# --- Yahoo Finance via yfinance: US tickers (real market yh_df) ---
tickers = ["SPY", "QQQ", "TLT", "GLD", "EEM"]
# SPY : S&P 500 index
# QQQ : Nasdaq-100 index
# TLT : U.S. Treasury bonds with 20+ year maturity
# GLD : Physical gold prices
# EEM : MSCI Emerging Markets index

try:
    import yfinance as yf
except Exception as e:
    yf = None
    print("Could not import yfinance:", type(e).__name__, str(e))

if yf is not None:
    try:
        yh_df = yf.download(tickers, start=START, end=END, auto_adjust=True, progress=False)
    except Exception as e:
        yh_df = pd.DataFrame()
        print("yfinance download failed:", type(e).__name__, str(e))
else:
    yh_df = pd.DataFrame()

# Convert to long format: date, ticker, close, volume
if isinstance(yh_df, pd.DataFrame) and yh_df.shape[0] > 0:
    if isinstance(yh_df.columns, pd.MultiIndex):
        close = yh_df["Close"].copy()
        vol = yh_df["Volume"].copy()
    else:
        close = yh_df[["Close"]].rename(columns={"Close": tickers[0]})
        vol = yh_df[["Volume"]].rename(columns={"Volume": tickers[0]})

    close.index.name = "date"
    vol.index.name = "date"

    us_close_long = close.reset_index().melt(id_vars="date", var_name="ticker", value_name="close")
    us_vol_long = vol.reset_index().melt(id_vars="date", var_name="ticker", value_name="volume")
    us_mkt = us_close_long.merge(us_vol_long, on=["date","ticker"], how="inner").dropna(subset=["close"])
else:
    us_mkt = pd.DataFrame(columns=["date","ticker","close","volume"])

us_mkt.head(), us_mkt.shape

(        date ticker      close    volume
 0 2022-01-03    EEM  44.624969  27572700
 1 2022-01-04    EEM  44.470776  24579500
 2 2022-01-05    EEM  43.745167  46425100
 3 2022-01-06    EEM  43.944710  34288700
 4 2022-01-07    EEM  44.343792  32640900,
 (4970, 4))

In [19]:
spy_df = us_mkt[us_mkt["ticker"] == "SPY"].copy()
spy_df

,date,ticker,close,volume
2982,2022-01-03,SPY,451.875153,72668200
2983,2022-01-04,SPY,451.723785,71178700
2984,2022-01-05,SPY,443.049713,104538900
2985,2022-01-06,SPY,442.633514,86858900
2986,2022-01-07,SPY,440.883545,85111600
...,...,...,...,...
3971,2025-12-11,SPY,687.139526,86173700
3972,2025-12-12,SPY,679.751404,113160300
3973,2025-12-15,SPY,678.724426,90811000
3974,2025-12-16,SPY,676.869934,122030600


In [46]:
spy_close_np = spy_df["close"].to_numpy()
spy_close_np

array([451.87515259, 451.7237854 , 443.04971313, 442.6335144 ,
       440.88354492, 440.33499146, 444.34564209, 445.54696655,
       439.40795898, 439.587677  , 431.80270386, 427.31903076,
       422.58947754, 414.29379272, 416.05319214, 410.97363281,
       409.94259644, 407.91827393, 418.04904175, 425.5786438 ,
       428.4541626 , 432.61621094, 422.44760132, 424.43405151,
       423.0718689 , 426.55285645, 432.79598999, 425.02050781,
       416.63964844, 415.27755737, 421.97470093, 422.44760132,
       413.42352295, 410.74658203, 406.33865356, 399.13070679,
       405.13720703, 414.07623291, 413.01681519, 406.72647095,
       414.2086792 , 412.14657593, 408.79797363, 396.74697876,
       393.73892212, 404.29537964, 402.46975708, 397.35238647,
       394.44836426, 403.12246704, 412.06137085, 417.21664429,
       421.78634644, 421.6630249 , 426.5970459 , 421.10317993,
       427.45098877, 429.53848267, 432.59387207, 437.94543457,
       435.24118042, 428.54217529, 429.75680542, 433.43

In [21]:
SPY_close_series = pd.Series(
    data=spy_close_np,
    index=spy_df["date"].values,
    name="SPY_close"
)
SPY_close_series

2022-01-03    451.875153
2022-01-04    451.723785
2022-01-05    443.049713
2022-01-06    442.633514
2022-01-07    440.883545
                 ...    
2025-12-11    687.139526
2025-12-12    679.751404
2025-12-15    678.724426
2025-12-16    676.869934
2025-12-17    669.421936
Name: SPY_close, Length: 994, dtype: float64

In [18]:
spy_mean = SPY_close_series.mean()
spy_min = SPY_close_series.min()
spy_max = SPY_close_series.max()

spy_mean, spy_min, spy_max

(np.float64(485.6085863631496), 341.1820373535156, 687.1395263671875)

In [22]:
last_closes = (
    us_mkt.sort_values("date")
          .groupby("ticker")["close"]
          .last()
          .to_dict()
)
last_closes

{'EEM': 52.599998474121094,
 'GLD': 399.2900085449219,
 'QQQ': 599.6373901367188,
 'SPY': 669.4219360351562,
 'TLT': 87.45963287353516}

In [23]:
last_close_series = pd.Series(last_closes).sort_values(ascending=False)

last_close_series

SPY    669.421936
QQQ    599.637390
GLD    399.290009
TLT     87.459633
EEM     52.599998
dtype: float64

In [42]:
spy_series = (
    us_mkt[us_mkt["ticker"] == "SPY"]
    .set_index("date")["close"]
    .rename("SPY_close")
)
spy_series

date
2022-01-03    451.875153
2022-01-04    451.723785
2022-01-05    443.049713
2022-01-06    442.633514
2022-01-07    440.883545
                 ...    
2025-12-11    687.139526
2025-12-12    679.751404
2025-12-15    678.724426
2025-12-16    676.869934
2025-12-17    669.421936
Name: SPY_close, Length: 994, dtype: float64

In [44]:
mid_rate_series = spy_series.rolling(5).mean().rename("mid_rate")
mid_rate_series

date
2022-01-03           NaN
2022-01-04           NaN
2022-01-05           NaN
2022-01-06           NaN
2022-01-07    446.033142
                 ...    
2025-12-11    683.799426
2025-12-12    683.015747
2025-12-15    682.437451
2025-12-16    681.605920
2025-12-17    678.381445
Name: mid_rate, Length: 994, dtype: float64

In [45]:
aligned_df = pd.DataFrame({
    "mid_rate": mid_rate_series,
    "SPY_close": spy_series
})

aligned_df.head()

,mid_rate,SPY_close
date,,
2022-01-03,NaN,451.875153
2022-01-04,NaN,451.723785
2022-01-05,NaN,443.049713
2022-01-06,NaN,442.633514
2022-01-07,446.033142,440.883545


In [28]:
# Versión NumPy (sin alineación)
n = min(len(spy_series), len(mid_rate_series))

spy_np = spy_series.values[:n]
mid_np = mid_rate_series.values[:n]

np.column_stack([spy_np, mid_np])[:5]

array([[451.87515259,          nan],
       [451.7237854 ,          nan],
       [443.04971313,          nan],
       [442.6335144 ,          nan],
       [440.88354492, 446.03314209]])

In [29]:
us_mkt_nan = us_mkt.copy()
us_mkt_nan

,date,ticker,close,volume
0,2022-01-03,EEM,44.624969,27572700
1,2022-01-04,EEM,44.470776,24579500
2,2022-01-05,EEM,43.745167,46425100
3,2022-01-06,EEM,43.944710,34288700
4,2022-01-07,EEM,44.343792,32640900
...,...,...,...,...
4965,2025-12-11,TLT,87.848114,26778700
4966,2025-12-12,TLT,87.001404,47030100
4967,2025-12-15,TLT,87.061180,28611800
4968,2025-12-16,TLT,87.539314,41018700


In [30]:
np.random.seed(42)

In [32]:
nan_idx = us_mkt_nan.sample(frac=0.01).index
us_mkt_nan.loc[nan_idx, "close"] = np.nan
nan_idx

Index([1558, 3057, 1249, 2891,   35, 1931,  388, 1568, 2515, 4572, 1682, 3299,
       4520, 1512, 3858, 2928, 4606, 2029, 2903, 2909, 3582,  671, 3781, 2328,
       3011, 1879,  483, 3171, 2522, 1100, 4909, 2124, 4307,  482, 1088,  107,
         39,  570, 3374, 1120, 3205,  150, 1446, 4906, 3772, 2285, 1155, 1084,
       2519, 1147],
      dtype='int64')

In [33]:
us_drop = us_mkt_nan.dropna(subset=["close"])
us_drop

,date,ticker,close,volume
0,2022-01-03,EEM,44.624969,27572700
1,2022-01-04,EEM,44.470776,24579500
2,2022-01-05,EEM,43.745167,46425100
3,2022-01-06,EEM,43.944710,34288700
4,2022-01-07,EEM,44.343792,32640900
...,...,...,...,...
4965,2025-12-11,TLT,87.848114,26778700
4966,2025-12-12,TLT,87.001404,47030100
4967,2025-12-15,TLT,87.061180,28611800
4968,2025-12-16,TLT,87.539314,41018700


In [34]:
us_fill = us_mkt_nan.copy()
median_by_ticker = us_fill.groupby("ticker")["close"].transform("median")
us_fill["close"] = us_fill["close"].fillna(median_by_ticker)

us_mkt.shape, us_drop.shape, us_fill.shape

((4970, 4), (4870, 4), (4970, 4))

In [35]:
dup_df = pd.concat([us_mkt.tail(5), us_mkt.tail(5)], ignore_index=True)
dup_df

,date,ticker,close,volume
0,2025-12-11,TLT,87.848114,26778700
1,2025-12-12,TLT,87.001404,47030100
2,2025-12-15,TLT,87.061180,28611800
3,2025-12-16,TLT,87.539314,41018700
4,2025-12-17,TLT,87.459633,24668300
5,2025-12-11,TLT,87.848114,26778700
6,2025-12-12,TLT,87.001404,47030100
7,2025-12-15,TLT,87.061180,28611800
8,2025-12-16,TLT,87.539314,41018700
9,2025-12-17,TLT,87.459633,24668300


In [36]:
dup_df["is_duplicate"] = dup_df.duplicated()

dup_df

,date,ticker,close,volume,is_duplicate
0,2025-12-11,TLT,87.848114,26778700,False
1,2025-12-12,TLT,87.001404,47030100,False
2,2025-12-15,TLT,87.061180,28611800,False
3,2025-12-16,TLT,87.539314,41018700,False
4,2025-12-17,TLT,87.459633,24668300,False
5,2025-12-11,TLT,87.848114,26778700,True
6,2025-12-12,TLT,87.001404,47030100,True
7,2025-12-15,TLT,87.061180,28611800,True
8,2025-12-16,TLT,87.539314,41018700,True
9,2025-12-17,TLT,87.459633,24668300,True


In [37]:
dup_clean = dup_df.drop_duplicates()

dup_clean.shape

(10, 5)

In [38]:
group_stats = (
    us_mkt
    .groupby("ticker")
    .agg(
        mean_close=("close", "mean"),
        median_close=("close", "median"),
        max_volume=("volume", "max")
    )
    .sort_values("mean_close", ascending=False)
)

group_stats

,mean_close,median_close,max_volume
ticker,,,
SPY,485.608586,460.740005,256611400
QQQ,411.276965,400.214539,198685800
GLD,220.130422,187.864998,62025000
TLT,91.395622,88.549713,131353500
EEM,40.462350,39.107409,134225700


In [39]:
last_close_df = pd.DataFrame(last_closes, index=["last_close"])

# Convertir a formato largo
last_close_long = last_close_df.melt(
    var_name="ticker",
    value_name="last_close"
)

last_close_long

,ticker,last_close
0,EEM,52.599998
1,GLD,399.290009
2,QQQ,599.637390
3,SPY,669.421936
4,TLT,87.459633


In [41]:
us_wide = (
    us_mkt
    .pivot(index="date", columns="ticker", values="close")
    .head(50)
)

us_wide.head(50)

ticker,EEM,GLD,QQQ,SPY,TLT
date,,,,,
2022-01-03,44.624969,168.330002,391.679413,451.875153,125.295387
2022-01-04,44.470776,169.570007,386.599213,451.723785,124.774399
2022-01-05,43.745167,169.059998,374.722382,443.049713,124.097084
2022-01-06,43.944710,166.990005,374.459106,442.633514,124.418343
2022-01-07,44.343792,167.750000,370.402679,440.883545,123.524071
2022-01-10,44.343792,168.259995,370.646423,440.334991,123.827957
2022-01-11,45.368717,170.289993,376.214294,444.345642,124.652802
2022-01-12,46.121532,170.740005,377.706177,445.546967,124.175247
2022-01-13,45.468483,170.160004,368.257416,439.407959,125.277969
